In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Classification avec Machine Learning classique\n",
    "\n",
    "Ce notebook illustre comment utiliser un algorithme de Machine Learning classique (Random Forest) pour classifier des chiffres manuscrits (dataset MNIST)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import des bibliothèques nécessaires\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import time\n",
    "from sklearn.ensemble import RandomForestClassifier\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import accuracy_score, confusion_matrix, classification_report\n",
    "from sklearn.decomposition import PCA\n",
    "import seaborn as sns"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Chargement et préparation des données MNIST"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Charger le dataset MNIST depuis scikit-learn\n",
    "from sklearn.datasets import fetch_openml\n",
    "mnist = fetch_openml('mnist_784', version=1, as_frame=False)\n",
    "X, y = mnist[\"data\"], mnist[\"target\"]\n",
    "X = X / 255.0  # Normalisation des valeurs de pixels entre 0 et 1\n",
    "y = y.astype(np.uint8)  # Conversion des labels en entiers\n",
    "\n",
    "# Afficher les dimensions des données\n",
    "print(f\"X shape: {X.shape}\")\n",
    "print(f\"y shape: {y.shape}\")\n",
    "print(f\"Classes: {np.unique(y)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Visualisation de quelques exemples"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Afficher quelques exemples d'images\n",
    "plt.figure(figsize=(10, 5))\n",
    "for i in range(10):\n",
    "    plt.subplot(2, 5, i+1)\n",
    "    plt.imshow(X[i].reshape(28, 28), cmap='gray')\n",
    "    plt.title(f\"Label: {y[i]}\")\n",
    "    plt.axis('off')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Réduction de dimension pour le Machine Learning classique\n",
    "\n",
    "Pour accélérer le traitement avec les algorithmes classiques, nous allons réduire la dimensionnalité des données avec PCA."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Utiliser un sous-ensemble des données pour la démonstration\n",
    "X_sample = X[:10000]\n",
    "y_sample = y[:10000]\n",
    "\n",
    "# Réduction de dimension avec PCA\n",
    "n_components = 50  # Réduire de 784 à 50 dimensions\n",
    "pca = PCA(n_components=n_components)\n",
    "X_reduced = pca.fit_transform(X_sample)\n",
    "\n",
    "print(f\"Forme originale: {X_sample.shape}\")\n",
    "print(f\"Forme après PCA: {X_reduced.shape}\")\n",
    "print(f\"Variance expliquée: {sum(pca.explained_variance_ratio_)*100:.2f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Séparation en ensembles d'entraînement et de test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Division en ensembles d'entraînement et de test\n",
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    X_reduced, y_sample, test_size=0.2, random_state=42\n",
    ")\n",
    "\n",
    "print(f\"X_train shape: {X_train.shape}\")\n",
    "print(f\"X_test shape: {X_test.shape}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Création et entraînement du modèle Random Forest"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Créer un modèle Random Forest\n",
    "n_estimators = 100  # Nombre d'arbres\n",
    "max_depth = 15      # Profondeur maximale des arbres\n",
    "\n",
    "rf_model = RandomForestClassifier(\n",
    "    n_estimators=n_estimators,\n",
    "    max_depth=max_depth,\n",
    "    random_state=42,\n",
    "    n_jobs=-1  # Utiliser tous les cœurs disponibles\n",
    ")\n",
    "\n",
    "# Mesurer le temps d'entraînement\n",
    "start_time = time.time()\n",
    "print(\"Entraînement du modèle Random Forest...\")\n",
    "rf_model.fit(X_train, y_train)\n",
    "end_time = time.time()\n",
    "training_time = end_time - start_time\n",
    "\n",
    "print(f\"Temps d'entraînement: {training_time:.2f} secondes\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Évaluation du modèle"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Évaluer le modèle sur les données de test\n",